# Fixed Point Precision Study on MLP-100 

This note book is the final stage of the model preparation for benchmarking. The next stage after this notebook is to simply implement the model in C using the knowledge gained from this notebook.  
The fixed-point operations defined here tries to simulate the computations performed in SPAR. This might change over-time.

**NOTE:**
- The programs/code-snippets in this notebook follows C-like interfaces on purpose.
- This is done so that, these code can be easily translated into C for the next stage of study.

# SQLite3 Database Utility Functions

In [1]:
import sqlite3

# Returns a list of table names in the database file
def getTableNames(db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Fetch the table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    table_names = cursor.fetchall()
    table_names = [name[0] for name in table_names]  # make a list to return
    conn.close()
    return table_names


# Returns a list of column names of the specified table
def getColNames(database_filename, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(database_filename)
    cursor = conn.cursor()

    # Fetch the column names
    cursor.execute(f"PRAGMA table_info({table_name})")
    results = cursor.fetchall()
    column_names = [result[1] for result in results]  # Extract the column names from the query results

    # Close the connection and return 
    conn.close()
    return column_names

    
# returns all records of a given table
def getRecords(db_path, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Fetch all records from the table
    cursor.execute(f"SELECT * FROM {table_name}")
    records = cursor.fetchall()
    return records


# Checks if a table exist
def existTable(db_path, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Check if the table exists
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
    result = cursor.fetchone()
    if result is None: exist = False
    else: exist = True
    # Commit the changes and close the connection and return result
    conn.commit()
    conn.close()
    return exist

# Load Floating-Point Model Parameters and Dataset

Here, the model and the dataset exported by the Numpy model extraction notebook is loaded and verified.

## Load the Dataset

In [2]:
# Load and check the dataset table
Dataset_path = './saved/mnist_test_data-98.36p.s3db'

table_names = getTableNames(Dataset_path)
print('table_names:', table_names)

# Read the header table
header_records =  getRecords(Dataset_path, 'Header')
header_dict = {}
print('')
for r in header_records: 
    print(r[1:3])
    header_dict[r[1]] = r[2]
    
# Get the table names
Data_table = header_dict['dataset.table']
Feature_table = header_dict['features.table']
Label_table = header_dict['labels.table']
print('')
print('Data_table:', Data_table)
print('Feature_table:', Feature_table)
print('Label_table:', Label_table)

table_names: ['Header', 'sqlite_sequence', 'Labels_T', 'DataItems_T', 'Features_T']

('name', 'MNIST-Test')
('feature_length', 784)
('accuracy', 98.36)
('labels.table', 'Labels_T')
('dataset.table', 'DataItems_T')
('dataitem.schema', '')
('features.table', 'Features_T')

Data_table: DataItems_T
Feature_table: Features_T
Label_table: Labels_T


In [3]:
# Read the label_to_index dictionary
labels_records = getRecords(Dataset_path, Label_table)
print('labels_records:', labels_records)

Label_to_index = {label:index for (index, label) in labels_records}
Index_to_label = {index:label for (index, label) in labels_records}
print('Label_to_index:', Label_to_index)
print('Index_to_label:', Index_to_label)

labels_records: [(0, '0'), (1, '1'), (2, '2'), (3, '3'), (4, '4'), (5, '5'), (6, '6'), (7, '7'), (8, '8'), (9, '9')]
Label_to_index: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}
Index_to_label: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9'}


### Build the Dataset array

In [4]:
import numpy as np
import dataclasses
from dataclasses import dataclass
from typing import List


# Dataset item class
@dataclass
class DataItem:
    label: str
    label_index: int
    predicted_index: int
    feature_vec: List[np.float32]
        
    def getItemSummary(self):
        return str((self.label, self.label_index, self.predicted_index, self.feature_vec.shape))

In [5]:
# Read the features and data-item records then merge them
# Make the feature_id:feature_vec map
feat_records = getRecords(Dataset_path, Feature_table)
featid_map = {}
for r in feat_records:
    feat_id = r[0]       # first column is the feature ID
    feat_vec = r[1:]
    featid_map[feat_id]  = feat_vec

# Read the data-items and put them in DataItem array
Dataset = []
data_records = getRecords(Dataset_path, Data_table)
data_schema = getColNames(Dataset_path, Data_table)
print('data_schema:', data_schema)

for r in data_records:
    label = r[1]
    label_index = r[2]
    pred_index = r[3]
    feat_id = r[4]
    feat_vec = np.array(featid_map[feat_id], dtype=np.float32)
    item = DataItem(label, label_index, pred_index, feat_vec)
    Dataset.append(item)

item = Dataset[0]
print('Dataset:', len(Dataset))
print('item:', item.getItemSummary())

data_schema: ['id', 'label', 'label_index', 'predicted_index', 'feature_id']
Dataset: 10000
item: ('7', 7, 7, (784,))


## Load the Trained Model Parameters

In [6]:
# Load and check the dataset table
Model_path = './saved/trained_mlp100-98.36p.s3db'

table_names = getTableNames(Model_path)
print('table_names:', table_names)

# Read the header table
header_records =  getRecords(Model_path, 'Header')
Header_dict = {}
print('')
for r in header_records: 
    print(r[1:3])
    Header_dict[r[1]] = r[2]



# Get the table names for later use
Fc1w_table = Header_dict['fc1.weight.table']
Fc1b_table = Header_dict['fc1.bias.table']
Fc2w_table = Header_dict['fc2.weight.table']
Fc2b_table = Header_dict['fc2.bias.table']
Fc3w_table = Header_dict['fc3.weight.table']
Fc3b_table = Header_dict['fc3.bias.table']

table_names: ['Header', 'sqlite_sequence', 'Hparam_T', 'FC1_Weight_T', 'FC2_Weight_T', 'FC3_Weight_T', 'FC1_Bias_T', 'FC2_Bias_T', 'FC3_Bias_T']

('name', 'MLP-100')
('architecture', '784-FC:100-FC:100-10')
('accuracy', 98.36)
('correct_count', 9836)
('Hparam.table', 'Hparam_T')
('fc1.weight.table', 'FC1_Weight_T')
('fc1.bias.table', 'FC1_Bias_T')
('fc2.weight.table', 'FC2_Weight_T')
('fc2.bias.table', 'FC2_Bias_T')
('fc3.weight.table', 'FC3_Weight_T')
('fc3.bias.table', 'FC3_Bias_T')


In [7]:
# Returns a table saved using createMatrixTable as a list of tuples
def readMatrixTable(db_path, table_name):
    # read the records
    rec_list = getRecords(db_path, table_name)
    # build the matrix
    rec_list.sort()         # sort by row_no (first column)
    matrix = []
    for rec in rec_list:
        matrix.append(rec[1:])  # stripe off the row_no columns
    return matrix


# test this functions
mat1 = np.array(readMatrixTable(Model_path, Fc1w_table))
mat1.shape

(100, 784)

In [8]:
# Returns the weights and biases as a dictionary
def readModelParam(db_path, table_names):
    model_params = {}
    for name in table_names:
        # read the matrix as a list of tuples
        mat = readMatrixTable(db_path, name)
        # Check if it is a matrix or a vector
        if len(mat)==1: is_vector = True
        else: is_vector = False
        # convert to numpy array
        if is_vector: mat = np.array(mat[0])    # make a 1D array for vectors
        else: mat = np.array(mat)
        # save it for returning
        model_params[name] = mat
    return model_params

# List of table names for running loop
ParamTable_names = [
    Fc1w_table,
    Fc1b_table,
    Fc2w_table,
    Fc2b_table,
    Fc3w_table,
    Fc3b_table
]
print('ParamTable_names:', ParamTable_names)

# Read the model parameters as numpy matrix/vectors
model_params = readModelParam(Model_path, ParamTable_names)
for k, v in model_params.items():
    print(f'{k}:', v.shape, v.dtype)

ParamTable_names: ['FC1_Weight_T', 'FC1_Bias_T', 'FC2_Weight_T', 'FC2_Bias_T', 'FC3_Weight_T', 'FC3_Bias_T']
FC1_Weight_T: (100, 784) float64
FC1_Bias_T: (100,) float64
FC2_Weight_T: (100, 100) float64
FC2_Bias_T: (100,) float64
FC3_Weight_T: (10, 100) float64
FC3_Bias_T: (10,) float64


In [9]:
# Model Parameters class
@dataclass
class mlp100_Params:
    fc1_weight: np.ndarray
    fc2_weight: np.ndarray
    fc3_weight: np.ndarray
    fc1_bias: np.ndarray
    fc2_bias: np.ndarray
    fc3_bias: np.ndarray
        

# Instantiate the model parameter class with float32 datatype
Model_params = mlp100_Params(
    model_params[Fc1w_table].astype(np.float32),
    model_params[Fc2w_table].astype(np.float32),
    model_params[Fc3w_table].astype(np.float32),
    model_params[Fc1b_table].astype(np.float32),
    model_params[Fc2b_table].astype(np.float32),
    model_params[Fc3b_table].astype(np.float32)
)

# Show the parameter info
for field in dataclasses.fields(Model_params):
    field_value = getattr(Model_params, field.name)
    print(field.name+':', field_value.shape, field_value.dtype)

fc1_weight: (100, 784) float32
fc2_weight: (100, 100) float32
fc3_weight: (10, 100) float32
fc1_bias: (100,) float32
fc2_bias: (100,) float32
fc3_bias: (10,) float32


# Verify the Model on the Dataset

In [10]:
# Relu on numpy array
def npReLU(np_arr):
    return np.maximum(0, np_arr)


# Performs the forward inference on the mlp100 model
# params: model parameters, an instance of mlp100_Params
def mlp100_forward(params, feature_vec):
    x1 = params.fc1_weight @ feature_vec + params.fc1_bias
    fc1_out = npReLU(x1)
    x2 = params.fc2_weight @ fc1_out + params.fc2_bias
    fc2_out = npReLU(x2)
    fc3_out = params.fc3_weight @ fc2_out + params.fc3_bias
    return fc3_out

# Uses the forward pass and converts the result into predicted_index
def mlp100_predict(params, feature_vec):
    out_vec = mlp100_forward(params, feature_vec)
    return np.argmax(out_vec)   # return the index of the highest probable class


# Test this model
item = Dataset[0]
pred = mlp100_predict(Model_params, item.feature_vec)
print('pred:', pred)
print(item.getItemSummary())

pred: 7
('7', 7, 7, (784,))


In [11]:
from tqdm.auto import tqdm


# Validate the Given model on the whole dataset
# model_params: instance of mlp100_Params
# model_predict: a function that takes model_params and a feacture_vec to compute predicted_index
def validateModel(model_params, model_predict):
    expect_miss = 0      # keeps track of no. of mismatche between prediction in dataset vs model prediction
    total_count = 0
    correct_count = 0
    for item in tqdm(Dataset):
        pred_index = model_predict(model_params, item.feature_vec)
        if pred_index != item.predicted_index: expect_miss += 1    # prediction does not match prediction in dataset
        if pred_index == item.label_index: correct_count += 1   # prediction matched the actual label-index
        total_count += 1
    # Compute and print statistics
    accuracy = (100.0 * correct_count) / total_count
    print(f'Validation accuracy: {accuracy:.2f}%   correct_count: {correct_count}   expected-miss: {expect_miss}   total_count: {total_count}')
    return accuracy, correct_count, expect_miss, total_count


validateModel(Model_params, mlp100_predict)
print(f'Expected:  accuracy: {Header_dict["accuracy"]}%', '  correct_count:', Header_dict['correct_count'])

  0%|          | 0/10000 [00:00<?, ?it/s]

Validation accuracy: 98.36%   correct_count: 9836   expected-miss: 0   total_count: 10000
Expected:  accuracy: 98.36%   correct_count: 9836


# Define Fixed-Point Methods

# Implement Model in Fixed-Point

# Experiments for Fixed-Point Precision

# Export Model and Dataset as SQLite3 DB